In [1]:
import requests
import datetime
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

user_agent = [
"Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
"Mozilla/5.0 (Windows NT 10.0; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
"Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
"Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)",
"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
"Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
"Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
"Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11",
"Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Avant Browser)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)"
]


def GET_PICTURE(url):
    b=requests.get(url,headers={'User-Agent':np.random.choice(user_agent)})
    #find header of url
    if bool(re.search(string=b.url,pattern='http')):
        addition_head=re.search(string=b.url,pattern='(http(s)*://[a-zA-Z0-9\.]+)/').group(1)
    else:
        addition_head=''
    b.encoding='utf-8'
    soup=BeautifulSoup(b.text,'lxml')
    #delete recommend nodes
    try:
        for s in soup.find_all(class_=re.compile('.*recommend.*|.*more.*|[^a-zA-Z]? ?related ?[^a-zA-Z ]+|[^a-zA-Z]+ ?related ?[^a-zA-Z ]?|^related$|.*aside.*')):
            s.decompose()
    except:
        soup=soup
    Result=[]
    #remove some annoyous fake image
    bad_words=['template','icon','logo','gif','/ad/']
    bad_pat=''
    for i,s in enumerate(bad_words):
        if i == 0:
            bad_pat=bad_pat+s
        else:
            bad_pat=bad_pat+'|'+s
    #find correct position of article
    N=0
    try:
        if soup.find('h1').text != '':
            article_body=soup.find('h1').find_parent()
        else:
            article_body=soup.find('h2').find_parent()
    except:
        try:
            article_body=soup.find('h2').find_parent()
        except:
            try:
                article_body=soup.find(class_=re.compile('.*title.*')).find_parent()
            except:
                N=100
                article_body='QQ,not exist h1 h2 anymore'
    #try to find correct article body
    try:
        while len(str(article_body))<5000 and N <20:
            article_body=article_body.find_parent()
            N=N+1
        if N == 100:
            raise RuntimeError('QQ,skip')
        article_body=article_body.find_all('img')
    except:
        try:
            article_body=soup.find_all('img')
        except:
            return('No pictures or can not deal with.')
    #find correct type of file of picture
    for s in article_body:
        try:
            node_name=re.search(string=str(s),pattern='\s*([-_a-zA-Z]*src)=').group(1)
            if bool(re.search(string=s.get(node_name),pattern='(jpg|jpeg|png)')) and not bool(re.search(string=s.get(node_name),pattern=bad_pat)):
                Result.append(s.get(node_name))
            else:
                continue
        except:
            continue
    if len(Result) == 0:
        for s in article_body:
            try:
                node_name=re.search(string=str(s),pattern='\s*([-_a-zA-Z]*src)=').group(1)
                if not bool(re.search(string=s.get(node_name),pattern=bad_pat)):
                    Result.append(s.get(node_name))
                else:
                    continue
            except:
                continue
    #try to return original url of picture
    if len(Result)>0:
        temp=[bool(re.search(string=s,pattern='^http')) for s in Result]
        if False in temp and True in temp:
            Result=Result
#             Result=[s for i,s in enumerate(Result) if not temp[i]]
        elif False not in temp and True in temp:
            return Result
        else:
            Result=Result
        #../../../n12377419/n12378191/c15390423/part/15390438.jpg
        #clean url in result
        not_complete=[]   #省略主要網址的關鍵網站
        seem_complete=[]  #省略了http or https 的網址
        for i,s in enumerate(Result):
            if bool(re.search(string=s,pattern='^//')):
                seem_complete.append(i)
                continue
            if bool(re.search(string=s,pattern='^http')):
                Result[i]=Result[i]
            else:
                s=re.sub(string=Result[i],pattern='\.\./',repl='')
                if not bool(re.search(string=s,pattern='^/')):
                    s='/'+s
                Result[i]=s
                not_complete.append(i)
        #處理省略了http or https 的網址
        if len(seem_complete)>0:
            for ii in seem_complete:
                if requests.get('https:'+Result[ii]).status_code != 200:
                    Result[ii]='http:'+Result[ii]
                else:
                    Result[ii]='https:'+Result[ii]
        #find correct addition_header 
        #prevent infinite loop
        if len(not_complete)>0:
            N=0
            while requests.get(addition_head+Result[not_complete[0]]).status_code != 200:
                if N == 10:
                    break
                addition_head=addition_head+'/'+re.sub(string=b.url,pattern=addition_head+'/',repl='').split('/')[0]
                N=N+1
            for s in not_complete:
                Result[s]=addition_head+Result[s]
        return Result
    else:
        return('No pictures or can not deal with.')
        

In [ ]:
GET_PICTURE('https://news.ltn.com.tw/news/life/breakingnews/3087051')

In [ ]:
requests.post('http://103.124.74.68:3266/GET_PICTURE',
              data={'url':'https://news.ltn.com.tw/news/life/breakingnews/3087051'}).json()

In [ ]:
GET_PICTURE('https://cn.reuters.com/article/japan-nov-industry-retail-sliding-1227-idCNKBS1YV061')

In [ ]:
GET_PICTURE('https://udn.com/news/story/120940/4404291')

In [ ]:
GET_PICTURE('https://www.ainonline.com/aviation-news/air-transport/2020-03-19/us-seeks-strike-balance-industry-pleads-bailouts')